# Stress Testing Facial Recognition with Adversarial Examples
By Emily Strong

## Introduction - The OpenFace Facial Recognition Model

## Adversarial Examples

## Validating the Pipeline
OpenFace comes with demo code for face classification. One of the examples included is of Steve Carell shown below. OpenFace uses older versions of several Python libraries so running the script returns warnings about depreciated features. I have suppressed these to make the output easier to read.

In [15]:
import warnings
warnings.simplefilter('ignore')

In [4]:
%run ./demos/classifier.py infer models/openface/celeb-classifier.nn4.small2.v1.pkl ./images/examples/carell.jpg


=== ./images/examples/carell.jpg ===
Predict SteveCarell with 0.99 confidence.


I can run this same script with a model I have trained on my own data to identify an image of Keira Knightley. In this project I am using two models, one trained only on the people used as subjects for my test cases, and one trained on the full FaceScrub data set.

In [5]:
%run ./demos/classifier.py infer ./generated-embeddings/limited/classifier.pkl test-images/keira_knightley3.jpg


=== test-images/keira_knightley3.jpg ===
Predict keira-knightley with 0.99 confidence.


In [6]:
%run ./demos/classifier.py infer ./generated-embeddings/fullfacescrub/classifier.pkl test-images/keira_knightley3.jpg


=== test-images/keira_knightley3.jpg ===
Predict keira-knightley with 0.81 confidence.


## Part A: Image Augmentation
In the first part of this project I am stress testing the classification model by augmenting the image in ways that should decrease the image quality or otherwise make it more difficult to obtain the measurements that OpenFace uses to identify the faces.

### Face Classification Workflow
I need to adapt the portions of "/demos/classifyer.py" used for testing a trained model so that it can be called within a while loop.

In [2]:
import argparse
import cv2
import os
import pickle
import sys

from operator import itemgetter

import numpy as np
np.set_printoptions(precision=2)
import pandas as pd

import openface

from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.grid_search import GridSearchCV
from sklearn.mixture import GMM
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

First set the paths for the models used in the pipeline

In [3]:
fileDir = os.path.dirname(os.path.realpath('FacialRecognition.ipynb'))
#modelDir = os.path.join(fileDir, '..', 'models')
modelDir = os.path.join(fileDir, 'models')
dlibModelDir = os.path.join(modelDir, 'dlib')
openfaceModelDir = os.path.join(modelDir, 'openface')

Check the path of the OpenFace model

In [4]:
openfaceModelDir

'/root/openface/models/openface'

In [18]:
align = openface.AlignDlib(os.path.join(dlibModelDir,"shape_predictor_68_face_landmarks.dat"))
net = openface.TorchNeuralNet(os.path.join(openfaceModelDir,'nn4.small2.v1.t7'), imgDim=96)
                           
def getRep(imgPath):

    bgrImg = cv2.imread(imgPath)
    if bgrImg is None:
        raise Exception("Unable to load image: {}".format(imgPath))

    rgbImg = cv2.cvtColor(bgrImg, cv2.COLOR_BGR2RGB)


    bb1 = align.getLargestFaceBoundingBox(rgbImg)
    
    if bb1 is None:
        raise Exception("Unable to find a face: {}".format(imgPath))
    bbs = [bb1]
    reps = []
    for bb in bbs:
        alignedFace = align.align(
            96,
            rgbImg,
            bb,
            landmarkIndices=openface.AlignDlib.OUTER_EYES_AND_NOSE)
        if alignedFace is None:
            raise Exception("Unable to align image: {}".format(imgPath))

        rep = net.forward(alignedFace)

        reps.append((bb.center().x, rep))
    sreps = sorted(reps, key=lambda x: x[0])
    return sreps

def infer(model, image):
    with open(model, 'rb') as f:
        if sys.version_info[0] < 3:
                (le, clf) = pickle.load(f)
        else:
                (le, clf) = pickle.load(f, encoding='latin1')

    
    print("\n=== {} ===".format(image))
    reps = getRep(image)
        
    for r in reps:
        rep = r[1].reshape(1, -1)
        bbx = r[0]
        predictions = clf.predict_proba(rep).ravel()
        maxI = np.argmax(predictions)
        person = le.inverse_transform(maxI)
        #confidence = predictions[maxI]
        #print("Predict {} with {:.2f} confidence.".format(person.decode('utf-8'), confidence))
        return person.decode('utf-8')

def run(model, image):
    output = infer(model, image)
    print(output)

In [20]:
run('generated-embeddings/fullfacescrub/classifier.pkl', 'test-images/colin_firth.jpg')


=== test-images/colin_firth.jpg ===
colin-firth


In [37]:
from imgaug import augmenters as iaa
from PIL import Image
import PIL.ImageOps    
from scipy.misc import imsave

In [43]:
def invert(model, image, name):
    output = infer(model, image)
    count = 0
    n = 0
    while(output==name and count<100):
        count+=1
        #seq = iaa.Sequential([iaa.Invert(n+.05, per_channel=False)])
        #temp = cv2.imread(image)
        #temp.load()
        #t = np.asarray(temp)
        #images_aug = seq.augment_images(temp)
        # https://github.com/aleju/imgaug/issues/38
        #for i, image_aug in enumerate(images_aug):
        #    image = 'test-images/' + name + str(count)+'.jpg'    
        #    imsave(image, image_aug)
        
        temp = Image.open(image)
        inverted_image = PIL.ImageOps.invert(temp)
        image = 'test-images/' + name + '-invert' + str(count)+'.jpg'
        inverted_image.save(image)
        output=infer(model, image)
    print(count)
        

In [41]:
invert('generated-embeddings/limited/classifier.pkl', 'test-images/colin_firth.jpg','colin-firth')


=== test-images/colin_firth.jpg ===

=== test-images/colin-firth-invert1.jpg ===


Exception: Unable to find a face: test-images/colin-firth-invert1.jpg

<img src="test-images/colin-firth-invert1.jpg" width="200"/>

In [46]:
invert('generated-embeddings/limited/classifier.pkl', 'test-images/george_lopez.jpg','george-lopez')


=== test-images/george_lopez.jpg ===

=== test-images/george-lopez-invert1.jpg ===
1


<img src="test-images/george-lopez-invert1.jpg" width="200"/>

In [47]:
run('generated-embeddings/fullfacescrub/classifier.pkl', 'test-images/george-lopez-invert1.jpg')


=== test-images/george-lopez-invert1.jpg ===
hayden-christensen
